# Tiago's Coursera Capstone Notebook

#### 1. Hello, this notebook is part of my journey at the Applied Data Science Capstone Project at Coursera. Find more at: https://www.coursera.org/learn/applied-data-science-capstone/home/welcome

### PART1

In [48]:
### 2. Imports
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import lxml
import requests

In [274]:
#retrive the webpage as requested.
pagina = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
resp = requests.get(pagina)

In [276]:
# Expecting 200 OK
resp


<Response [200]>

In [324]:
# Parsing with BeatifulSoup
soup = BeautifulSoup(resp.text, 'lxml')
#print(soup.table.text)
#print(soup.table)

In [325]:
#soup.table

In [326]:
#print(soup.prettify())
df = pd.DataFrame
#df = soup.table.text
df = pd.read_html(str(soup.table))
df = df[0]

In [327]:
#removing "Not assigned Borough's"
df = df.loc[:][df['Borough']!='Not assigned'] 
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [328]:
df.loc[:][df['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


In [329]:
# Setting Neighbourhood = Borough, if Neighbourhood = 'Not assigned'
df.loc[df['Neighbourhood'] == 'Not assigned',['Borough','Neighbourhood']]  = df['Borough']


In [330]:
# Expecting a null list
df.loc[:][df['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood


In [331]:
df.shape

(211, 3)

In [332]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [333]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda tags: ','.join(tags))

In [334]:
df = pd.DataFrame(df)

In [335]:
df.head()

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [336]:
#Finally, the DataFrame expected
df.reset_index(inplace = True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## 

In [338]:
df[df.loc[:]['Postcode'] == 'M5A']

,Postcode,Borough,Neighbourhood
53,M5A,Downtown Toronto,"Harbourfront,Regent Park"


In [339]:
df.shape

(103, 3)

### END OF PART1 

### PART2

In [369]:
df['lat'] = df['long'] = np.nan

In [441]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")

for index, row in df.iterrows():
    location = str(df.loc[index]['Borough'] +',' + df.loc[index]['Postcode']+', Canada')
    result = geolocator.geocode(location)
    try:
        df['lat'].loc[index] = result.latitude
        df['long'].loc[index] = result.longitude
    except:
        pass

In [447]:
df.head()

,Postcode,Borough,Neighbourhood,lat,long
0,M1B,Scarborough,"Rouge,Malvern",43.773077,-79.257774
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.773077,-79.257774
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",NaN,NaN
3,M1G,Scarborough,Woburn,43.762669,-79.230861
4,M1H,Scarborough,Cedarbrae,NaN,NaN


#### As seen above, geocoders could not convert all zip-codes + Boroughs to geographical coordinates. So, I had
#### to use the CSV file provided. 

In [452]:
df_coord = pd.read_csv('Geospatial_Coordinates.csv')

In [454]:
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [457]:
### Using the merge procedure to grab coordinates from postal codes.
df = pd.merge(left = df, right = df_coord, how='left', left_on='Postcode', right_on = 'Postal Code')

In [459]:
df.head()

,Postcode,Borough,Neighbourhood,lat,long,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.773077,-79.257774,M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.773077,-79.257774,M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",NaN,NaN,M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.762669,-79.230861,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,NaN,NaN,M1H,43.773136,-79.239476


In [461]:
df.drop(['lat', 'long', 'Postal Code'], axis = 1, inplace=True)

### Final Result

In [464]:
### RESULT
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [466]:
df.shape

(103, 5)